# 🔬 Multi-Agent RAG Research Assistant

This notebook demonstrates a multi-agent RAG (Retrieval-Augmented Generation) pipeline using LangGraph, FAISS, Together.ai (LLaMA-4), and optional PDF ingestion.

Users can interactively generate summaries from either:
- Web search results (via SerpAPI), or
- Uploaded research PDFs

It includes a modular design, agent-based orchestration, and a Gradio UI for live summarization based on user-defined prompts.


## ⚙️ Tech Stack

- **LangGraph** – Agent orchestration and graph state management  
- **FAISS** – Vector indexing and similarity retrieval  
- **MiniLM (sentence-transformers)** – Document embedding  
- **Together.ai** – LLM-powered summarization (LLaMA-4)  
- **SerpAPI** – Google search result ingestion  
- **PyMuPDF** – PDF parsing and text extraction  
- **Gradio** – Interactive user interface


In [1]:
# Basic NLP and retrieval tools
!pip install -q sentence-transformers langchain faiss-cpu serpapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.8 MB/s eta 0:00:00


In [2]:
# LLaMA model loader (this is the slow one that may need C++ compilation)
!pip install -q llama-cpp-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 11.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.0 MB/s eta 0:00:00


In [4]:
!pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.50
    Uninstalling langchain-core-0.3.50:
      Successfully uninstalled langchain-core-0.3.50
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.22
    Uninstalling langchain-0.3.22:
      Successfully uninstalled langchain-0.3.22


In [5]:
!pip install llama-cpp-python[server]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.4 MB/s eta 0:00:00


In [6]:
!pip install together


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 4.2 MB/s eta 0:00:00


In [7]:
!pip install langgraph


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.4/142.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 14.4 MB/s eta 0:00:00


In [8]:
!pip install grandalf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.7 MB/s eta 0:00:00


In [9]:
!pip install gradio
!python app.py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 60.2 MB/s eta 0:00:00
python3: can't open file '/content/app.py': [Errno 2] No such file or directory


In [10]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 19.5 MB/s eta 0:00:00


In [11]:
import gradio as gr
import tempfile
import os
import requests
import faiss
import numpy as np
from typing import TypedDict, List, Optional

from langgraph.graph import StateGraph
from langchain_core.runnables import RunnableLambda
from langchain.docstore.document import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.document_loaders import PyMuPDFLoader
from together import Together

##Agents##

## 🧪 Pipeline Flow

1. 🔍 **SearchAgent** – Uses SerpAPI to fetch top Google results
2. 📄 **PDFLoaderAgent** – Loads and splits uploaded PDFs into text
3. 🧠 **EmbedAgent** – Embeds all documents using MiniLM
4. 🔎 **RetrieveAgent** – Uses FAISS to find top-k relevant chunks
5. 📝 **SummarizeAgent** – Calls Together.ai LLaMA-4 with user prompt

The entire workflow is managed via LangGraph's stateful graph execution.


In [12]:

def search_agent(state):
    print("🔍 [SearchAgent] Fetching from SerpAPI...")
    query = state.get("query", "")
    api_key = state.get("serpapi_key", "")
    if not query or not api_key:
        print("⚠️ [SearchAgent] Missing query or API key.")
        return state

    params = {"engine": "google", "q": query, "api_key": api_key, "num": 20}
    resp = requests.get("https://serpapi.com/search", params=params)
    data = resp.json()
    snippets = [item.get("snippet", "") for item in data.get("organic_results", []) if item.get("snippet")]
    docs = [Document(page_content=s, metadata={"source": "serpapi", "rank": i}) for i, s in enumerate(snippets)]

    state["documents"] = docs  # Replace any existing docs
    print(f"🔍 [SearchAgent] Retrieved {len(docs)} snippets.")
    return state

In [13]:
def pdf_loader_agent(state):
    print("📄 [PDFLoaderAgent] Loading PDF...")
    pdf_path = state.get("pdf_path")

    # Ensure 'documents' key exists
    if "documents" not in state:
        state["documents"] = []

    if pdf_path and os.path.exists(pdf_path):
        loader = PyMuPDFLoader(pdf_path)
        pdf_docs = loader.load()
        state["documents"].extend(pdf_docs)
        print(f"📄 [PDFLoaderAgent] Loaded {len(pdf_docs)} pages from PDF.")
    else:
        print("⚠️ [PDFLoaderAgent] No PDF found.")
    return state

In [14]:
def embed_agent(state):
    print("🧠 [EmbedAgent] Embedding and indexing...")
    docs = state["documents"]
    embed_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectors = embed_model.embed_documents([doc.page_content for doc in docs])
    vectors_np = np.array(vectors).astype("float32")
    d = vectors_np.shape[1]
    nlist = min(5, len(docs))
    quantizer = faiss.IndexFlatL2(d)
    index = faiss.IndexIVFFlat(quantizer, d, nlist)
    index.train(vectors_np)
    index.add(vectors_np)

    index_to_docstore_id = {i: str(i) for i in range(len(docs))}
    docstore = InMemoryDocstore({str(i): doc for i, doc in enumerate(docs)})
    vectorstore = FAISS(embed_model, index, docstore, index_to_docstore_id)
    state["vectorstore"] = vectorstore
    print("✅ [EmbedAgent] Indexing complete.")
    return state

In [15]:
def retrieve_agent(state):
    print("🔎 [RetrieveAgent] Retrieving relevant documents...")
    vectorstore = state["vectorstore"]
    query = state.get("query", "")
    top_k = state.get("top_k", 3)
    results = vectorstore.similarity_search(query, k=top_k)
    state["retrieved_docs"] = results
    print(f"🔎 [RetrieveAgent] Retrieved {len(results)} docs.")
    return state


In [16]:
def summarize_agent(state):
    print("📝 [SummarizeAgent] Generating summary (based on user instructions)...")

    client = Together(api_key=state["together_api_key"])
    retrieved_texts = [doc.page_content for doc in state["retrieved_docs"]]

    user_query = state["query"]

    prompt = (
        f"You are a helpful AI research assistant.\n\n"
        f"The user has asked: \"{user_query}\"\n\n"
        f"Below is the retrieved research content. Please summarize it according to the user's request.\n\n"
        f"### Research Content:\n\n" + "\n\n".join(retrieved_texts)
    )

    response = client.chat.completions.create(
        model="meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8",
        messages=[
            {"role": "system", "content": "You are a helpful scientific summarizer."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=4096,  # allow long summaries
        temperature=0.7,
    )

    summary = response.choices[0].message.content
    print("✅ [SummarizeAgent] Summary generated.")
    return {**state, "summary": summary}


##LangGraph definition##

## 🧪 Pipeline Architecture

[User Query / PDF Upload] ↓ SearchAgent 🔍 (via SerpAPI) ↓ PDFLoaderAgent 📄 (if PDF exists) ↓ EmbedAgent 🧠 (MiniLM + FAISS) ↓ RetrieveAgent 🔎 (Top-k retrieval) ↓ SummarizeAgent ✍️ (Together.ai LLaMA-4) ↓ [Summary Output 📝]



## 📝 How to Use

1. Enter a query in the Gradio interface (e.g., "AutoML in 2024, 500-word summary")
2. Optionally upload a research PDF file
3. Provide your API keys:
   - `SerpAPI Key` (for Google search)
   - `Together.ai Key` (for LLaMA-4 summarization)
4. Click Submit and wait for the generated response


## 🔐 API Keys Required

- **SerpAPI Key**: [Get from serpapi.com](https://serpapi.com)
- **Together API Key**: [Get from platform.together.xyz](https://platform.together.xyz)

Keys can be entered directly into the Gradio app fields, or managed via `.env` (if running locally).


In [17]:
class RAGState(TypedDict):
    query: str
    serpapi_key: str
    together_api_key: str
    top_k: int
    pdf_path: Optional[str]
    documents: List[Document]
    vectorstore: FAISS
    retrieved_docs: List[Document]
    summary: str

graph = StateGraph(state_schema=RAGState)
graph.add_node("SearchAgent", RunnableLambda(search_agent))
graph.add_node("PDFLoaderAgent", RunnableLambda(pdf_loader_agent))
graph.add_node("EmbedAgent", RunnableLambda(embed_agent))
graph.add_node("RetrieveAgent", RunnableLambda(retrieve_agent))
graph.add_node("SummarizeAgent", RunnableLambda(summarize_agent))

graph.set_entry_point("SearchAgent")
graph.add_edge("SearchAgent", "PDFLoaderAgent")
graph.add_edge("PDFLoaderAgent", "EmbedAgent")
graph.add_edge("EmbedAgent", "RetrieveAgent")
graph.add_edge("RetrieveAgent", "SummarizeAgent")
graph.set_finish_point("SummarizeAgent")

dag = graph.compile()

##Gradio Interface##

## 💡 Example Prompts

- "Summarize this PDF in 500 words with Introduction, Methodology, and Results"
- "What are the major NAS techniques in 2024? Give bullet points"
- "List pros and cons of AutoML methods from this paper"


## ✅ Sample Output (Truncated)

- The paper introduces a hybrid NAS method combining reinforcement learning and gradient descent.
- It compares 3 different architectures: X, Y, Z...
- Key metrics: Accuracy = 92.4%, Latency = 0.8ms


## 📊 Results

- Achieved summaries of up to 1000 words respecting structural prompts
- Enabled hybrid PDF + Search summarization
- Fast similarity search over embedded corpus using IVF-indexed FAISS


In [18]:

def run_rag_pipeline(query, serpapi_key, together_api_key, pdf_file):
    try:
        print("\n🚀 Starting RAG pipeline...")
        pdf_path = None

        if pdf_file is not None:
            pdf_path = pdf_file.name
            print(f"📄 Using uploaded PDF at: {pdf_path}")

        state = {
            "query": query if query else "placeholder",
            "serpapi_key": serpapi_key,
            "together_api_key": together_api_key,
            "top_k": 3,
            "pdf_path": pdf_path,
        }

        output = dag.invoke(state)

        if not output.get("summary"):
            return "⚠️ No summary generated. Check if the PDF or search returned useful content."

        return output["summary"]

    except Exception as e:
        import traceback
        traceback.print_exc()
        return f"❌ Error: {str(e)}"


In [19]:
# Launch Gradio app
gr.Interface(
    fn=run_rag_pipeline,
    inputs=[
        gr.Textbox(label="📌 Research Query", placeholder="e.g., NAS techniques in 2024"),
        gr.Textbox(label="🔑 SerpAPI Key", type="password"),
        gr.Textbox(label="🔐 Together API Key", type="password"),
        gr.File(label="📄 Upload Research PDF (Optional)", file_types=[".pdf"])
    ],
    outputs=gr.Textbox(label="📝 Summary Output", lines=15),
    title="🔬 Research Assistant Chatbot (LangGraph + LLaMA)",
    description="Combines SerpAPI + PDF + FAISS + Together.ai for autonomous research summaries."
).launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://29b781b604c90ad629.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
